# Question 1

## Importing libraries

In [110]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Reading and cleaning dataset

In [173]:
# read the dataset
dataset = pd.read_csv('C:/Users/Asus/Downloads/cereal.csv', sep=';', skiprows=[1])

# set the X and y values
X = dataset.drop('name', axis=1)
X = X.drop('rating', axis=1)
y = dataset.iloc[:, -1].values


# one-hot encode the dataset for dataset['type'] and dataset['mfr']

# explore mfr column
print("Exploring mfr column\n", X['mfr'].value_counts())

# explore the type column
print("\nExploring type column\n", X['type'].value_counts())

Encoder = LabelEncoder()

for column in X:
    X[column] = Encoder.fit_transform(X[column])
    X[column].unique()

# display encoded dataset:
display(X)



Exploring mfr column
 K    23
G    22
P     9
Q     8
R     8
N     6
A     1
Name: mfr, dtype: int64

Exploring type column
 C    74
H     3
Name: type, dtype: int64


,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups
0,3,0,1,3,1,9,11,1,7,33,1,2,2,1
1,5,0,6,2,4,1,3,3,9,24,0,2,2,7
2,2,0,1,3,1,23,10,2,6,34,1,2,2,1
3,2,0,0,3,0,11,12,3,1,35,1,2,2,2
4,6,0,5,1,2,17,1,12,9,0,1,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,1,0,5,1,1,22,0,19,4,10,1,2,2,4
73,1,0,5,0,1,11,0,10,13,3,1,1,2,7
74,6,0,4,2,1,20,6,15,4,20,1,0,2,3
75,1,0,4,2,1,17,6,15,4,19,1,0,2,7


## Getting train-test split 80-20

In [72]:
# get train and test set with 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## 1.a)Performing Linear Regression with sklearn

In [73]:
# train the model
rating_predictor = LinearRegression()
rating_predictor.fit(X_train, y_train)

LinearRegression()

### Displaying coefficients and r-squared

In [112]:
# predict on test data
y_pred = rating_predictor.predict(X_test)
model_coef = rating_predictor.coef_ 
model_rsq = r2_score(y_test, y_pred)
# display the coefficients
print("coefficients:", model_coef)
print("\nr-squared:", model_rsq)

coefficients: [-0.07511833 -3.69243675 -2.93617256  3.70797205 -1.8372835  -0.4533807
  1.87055594  0.8641529  -0.57773473 -0.22979635 -4.30391171  0.73034643
 -0.06953334 -0.18416742]

r-squared: 0.885418166586389


### Comparison of actual rating vs predicted rating

In [100]:
# add the predictions to X and display the new dataframe
print(y_pred)
predicted_ratings = pd.DataFrame(y_pred, columns=['prediction'])
predicted_ratings = pd.concat([dataset['rating'], predicted_ratings], axis=1)
predicted_ratings = predicted_ratings.fillna(0)
display(predicted_ratings)

[31.27311972 51.20999518 39.9174466  62.24099352 46.68732852 58.56336907
 59.37395076 56.14380388 33.81900107 38.40191703 40.6593119  54.44504539
 73.66546064 28.1219168  49.56922812 35.96767297]


,rating,prediction
0,68.402973,31.273120
1,33.983679,51.209995
2,59.425505,39.917447
3,93.704912,62.240994
4,34.384843,46.687329
...,...,...
72,39.106174,0.000000
73,27.753301,0.000000
74,49.787445,0.000000
75,51.592193,0.000000


## 1.b) Calculating coefficients using numpy

In [96]:
def estimate_coef_multiple_linear_regression(X, y): 
    return np.dot(np.linalg.inv(np.dot(X.T,X)),np.dot(X.T,y))

fn_coef = estimate_coef_multiple_linear_regression(X, y)

## 1.c)  Comparing model coefficients with function coefficients

In [97]:
coefficients1 = pd.DataFrame(fn_coef, columns=['function'])
coefficients2 = pd.DataFrame(model_coef, columns=['model'])
coefficients = pd.concat([coefficients2, coefficients1], axis=1)
display(coefficients)

,model,function
0,-0.075118,1.200248
1,-3.692437,16.310597
2,-2.936173,-8.685891
3,3.707972,7.002815
4,-1.837283,3.048444
5,-0.453381,-0.197545
6,1.870556,2.650550
7,0.864153,3.203117
8,-0.577735,2.447856
9,-0.229796,-0.015181


## 1.d) Function to calculate r-squared using numpy

In [113]:
def rSquared(y_prediction, y):
    correlationMatrix = np.corrcoef(y_prediction, y)
    correlation = correlationMatrix[0,1]
    rsq = correlation**2
    return rsq

fn_rsq = rSquared(y_test, y_pred)

## 1.e) Comparing r-squared

In [114]:
print("R-Squared model value:\n", model_rsq, "\nR-Squared function value:\n", fn_rsq)

R-Squared model value:
 0.885418166586389 
R-Squared function value:
 0.9187001337644907


# Question 2

In [166]:
# import dataset

filename = 'C:/Users/Asus/Downloads/Caravan.csv'
data = pd.read_csv(filename)
Y2 = data.Purchase
X2 = data.drop(['Purchase'], axis=1)
display(X2)
display(Y2)

,Unnamed: 0,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,...,ALEVEN,APERSONG,AGEZONG,AWAOREG,ABRAND,AZEILPL,APLEZIER,AFIETS,AINBOED,ABYSTAND
0,1,33,1,3,2,8,0,5,1,3,...,0,0,0,0,1,0,0,0,0,0
1,2,37,1,2,2,8,1,4,1,4,...,0,0,0,0,1,0,0,0,0,0
2,3,37,1,2,2,8,0,4,2,4,...,0,0,0,0,1,0,0,0,0,0
3,4,9,1,3,3,3,2,3,2,4,...,0,0,0,0,1,0,0,0,0,0
4,5,40,1,4,2,10,1,4,1,4,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5817,5818,36,1,1,2,8,0,6,1,2,...,2,0,0,0,1,0,0,0,0,0
5818,5819,35,1,4,4,8,1,4,1,4,...,0,0,0,0,1,0,0,0,0,0
5819,5820,33,1,3,4,8,0,6,0,3,...,0,0,0,0,1,0,0,0,0,0
5820,5821,34,1,3,2,8,0,7,0,2,...,0,0,0,0,0,0,0,0,0,0


0        No
1        No
2        No
3        No
4        No
       ... 
5817     No
5818     No
5819    Yes
5820     No
5821     No
Name: Purchase, Length: 5822, dtype: object

In [167]:
# splitting the dataset 

X_train, X_test, y_train, y_test = train_test_split(X2, Y2, test_size=0.2, random_state=0)

In [168]:
# Feature scaling
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [169]:
# train the model (logistic regression)
from sklearn.linear_model import LogisticRegression

log_reg_model = LogisticRegression(solver = 'liblinear', class_weight='balanced').fit(X_train, y_train)

In [165]:
# predict the results of the model
y_pred = log_reg_model.predict(X_test)
print(y_pred)

['Yes' 'Yes' 'Yes' ... 'Yes' 'Yes' 'Yes']


In [148]:
# Show confustion matrix
from sklearn.metrics import confusion_matrix

conf_matr = confusion_matrix(y_test, y_pred)
print(conf_matr)

[[1093    3]
 [  67    2]]


In [149]:
# Show accuracy
from sklearn.metrics import accuracy_score

log_accuracy = accuracy_score(y_test, y_pred)
print(log_accuracy)

0.9399141630901288


## Comparing dataset values with prediction

In [171]:
model_pred = pd.DataFrame(y_pred, columns=['prediction'])
data_val = pd.DataFrame(y_test.values, columns=['dataset'])
comparison = pd.concat([data_val, model_pred], axis=1)
display(comparison)

,dataset,prediction
0,No,Yes
1,No,Yes
2,No,Yes
3,No,Yes
4,No,Yes
...,...,...
1160,No,Yes
1161,No,Yes
1162,No,Yes
1163,No,Yes


### It can be inferred from above that a new customer is predicted to purchase Caravan Insurace based on the prediction values and an accuracy of 93%